In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns

import os
import time

from tqdm import tqdm

#Data loading
Odaberi zmedu 100k i 25M

In [ ]:
!wget -nc http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip -n ml-latest-small.zip
# Loading datasets
movies_df = pd.read_table('ml-latest-small/movies.csv', sep=',')
ratings_df = pd.read_csv('ml-latest-small/ratings.csv')
tags_df = pd.read_csv('ml-latest-small/tags.csv')



--2023-07-06 10:14:38--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  --.-KB/s    in 0.1s    

2023-07-06 10:14:39 (6.64 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [ ]:
!wget -nc http://files.grouplens.org/datasets/movielens/ml-25m.zip
!unzip -n ml-25m.zip
# Loading datasets
movies_df = pd.read_table('ml-25m/movies.csv', sep=',')
ratings_df = pd.read_csv('ml-25m/ratings.csv')

File ‘ml-25m.zip’ already there; not retrieving.

Archive:  ml-25m.zip


In [ ]:
# Displaying the shape of each DataFrame
print("movies_df shape:", movies_df.shape)
print("ratings_df shape:", ratings_df.shape)

movies_df shape: (9742, 3)
ratings_df shape: (100836, 4)


#Reduciraj dataset

#Pripremi merged

In [ ]:
# Merging movies and ratings dataframes on movieId
merged_df = pd.merge(movies_df, ratings_df, on='movieId')
merged_df

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483
...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184,4.0,1537109082
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184,3.5,1537109545
100833,193585,Flint (2017),Drama,184,3.5,1537109805
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,3.5,1537110021


In [ ]:
# Convert data types
merged_df['movieId'] = merged_df['movieId'].astype('uint32')
merged_df['userId'] = merged_df['userId'].astype('uint32')
merged_df['rating'] = merged_df['rating'].astype('float32')
merged_df.drop(["timestamp"], axis=1, inplace=True)
merged_df.drop(["title"], axis=1, inplace=True)

In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   movieId  100836 non-null  uint32 
 1   genres   100836 non-null  object 
 2   userId   100836 non-null  uint32 
 3   rating   100836 non-null  float32
dtypes: float32(1), object(1), uint32(2)
memory usage: 2.7+ MB


In [ ]:
user_movie_counts = ratings_df.groupby('userId')['movieId'].count().reset_index()

user_movie_counts.columns = ['userId', 'movieCount']

quartiles = user_movie_counts['movieCount'].quantile([0.25, 0.5, 0.75])
n = 1000
q0, q1, q2, q3 = 0, quartiles.loc[0.25], quartiles.loc[0.5], quartiles.loc[0.75]

selected_users = user_movie_counts[
    (user_movie_counts['movieCount'] >= q0) &
    (user_movie_counts['movieCount'] < q1)
].sample(n).append(
    user_movie_counts[
        (user_movie_counts['movieCount'] >= q1) &
        (user_movie_counts['movieCount'] < q2)
    ].sample(n)
).append(
    user_movie_counts[
        (user_movie_counts['movieCount'] >= q2) &
        (user_movie_counts['movieCount'] < q3)
    ].sample(n)
).append(
    user_movie_counts[
        (user_movie_counts['movieCount'] >= q3)
    ].sample(n)
)

selected_user_ids = selected_users['userId'].tolist()

# Filter the 'merged_df' based on the selected user IDs
selected_users = merged_df[merged_df['userId'].isin(selected_user_ids)]

selected_users

ValueError: ignored

In [ ]:
merged_df=selected_users
merged_df

,movieId,genres,userId,rating
79,1,Adventure|Animation|Children|Comedy|Fantasy,265,4.0
86,1,Adventure|Animation|Children|Comedy|Fantasy,301,3.0
109,1,Adventure|Animation|Children|Comedy|Fantasy,370,3.0
156,1,Adventure|Animation|Children|Comedy|Fantasy,506,3.0
188,1,Adventure|Animation|Children|Comedy|Fantasy,598,1.0
...,...,...,...,...
24999879,208497,Documentary,129375,1.0
24999881,208499,Comedy|Documentary,129375,3.0
24999882,208501,(no genres listed),129375,2.0
24999944,208715,Comedy|Romance,113752,3.0


#Stvori users_df

In [ ]:
np.random.seed(271)

genres_df = merged_df.assign(genres=merged_df['genres'].str.split('|')).explode('genres')

user_genre_df = genres_df.groupby(['userId', 'genres']).agg({'rating': ['count', 'mean']}).reset_index()
user_genre_df.columns = ['userId', 'genres', 'movies_count', 'avg_rating']

user_genre_df['weighted_rating'] = user_genre_df['movies_count'] * user_genre_df['avg_rating']

users_df = user_genre_df.loc[user_genre_df.groupby('userId')['weighted_rating'].idxmax()].reset_index(drop=True)
users_df = users_df.drop(['movies_count', 'avg_rating', 'weighted_rating'], axis=1)

age_distribution = pd.DataFrame({
    'genres': ['Animation', 'Comedy', 'Adventure', 'Crime', 'Action', 'Sci-fi', 'Drama', 'Romance',
               'Thriller', 'Horror', 'Documentary', 'Children', 'Mystery', 'Fantasy', 'War', 'Western',
               'Musical', '(no genres listed)'],
    'Teenagers (13-19)': [55, 50, 54, 43, 63, 63, 35, 53, 63, 63, 40, 60, 63, 63, 45, 40, 60, 50],
    'Young Adults (20-25)': [25, 20, 19, 28, 17, 17, 20, 18, 17, 19, 20, 10, 17, 17, 15, 15, 20, 20],
    'Adults (26-35)': [5, 15, 14, 13, 12, 12, 15, 10, 12, 12, 15, 11, 12, 12, 15, 15, 10, 15],
    'Middle-aged Adults (36-45)': [10, 10, 9, 8, 6, 6, 15, 10, 6, 5, 15, 17, 6, 6, 15, 20, 5, 10],
    'Senior Adults (46-65)': [5, 5, 4, 8, 2, 2, 15, 9, 2, 1, 10, 2, 2, 2, 10, 10, 5, 5]
})


age_distribution.iloc[:, 1:] = age_distribution.iloc[:, 1:].div(age_distribution.iloc[:, 1:].sum(axis=1), axis=0)
merged_result_age = pd.merge(users_df, age_distribution, on='genres')

age_ranges = {
    'Teenagers (13-19)': (13, 19),
    'Young Adults (20-25)': (20, 25),
    'Adults (26-35)': (26, 35),
    'Middle-aged Adults (36-45)': (36, 45),
    'Senior Adults (46-65)': (46, 65)
}

def assign_age(row):
    age_group = np.random.choice(merged_result_age.columns[2:7], p=list(row[2:7]))

    return np.random.randint(age_ranges[age_group][0], age_ranges[age_group][1] + 1)

merged_result_age['age'] = merged_result_age.apply(assign_age, axis=1)
merged_result_age['age'] = merged_result_age.apply(assign_age, axis=1)

# Create an age group feature based on the age assigned to each user
def age_to_age_group(age):
    for group, age_range in age_ranges.items():
        if age_range[0] <= age <= age_range[1]:
            return group
    return None

merged_result_age['age_group'] = merged_result_age['age'].apply(age_to_age_group)

# Final DataFrame containing the users, their favorite genres, and their assigned ages and age groups
users_df = merged_result_age[['userId', 'genres', 'age', 'age_group']]

sex_values = np.random.choice(['M', 'F'], size=len(users_df), p=[0.65, 0.35])
users_df['sex'] = sex_values

country_populations = {
    'Russia': 145934462, 'Germany': 83783942, 'United Kingdom': 67886011, 'France': 65273511,
    'Italy': 60461826, 'Spain': 46754778, 'Ukraine': 43733762, 'Poland': 37846611,
    'Romania': 19237691, 'Netherlands': 17134872, 'Belgium': 11589623, 'Czech Republic': 10708981,
    'Greece': 10423054, 'Portugal': 10196709, 'Sweden': 10099265, 'Hungary': 9660351,
    'Belarus': 9449323, 'Austria': 9006398, 'Serbia': 8737371, 'Switzerland': 8654622,
    'Bulgaria': 6948445, 'Denmark': 5792202, 'Finland': 5540720, 'Slovakia': 5459642,
    'Norway': 5421241, 'Ireland': 4937786, 'Croatia': 4105267, 'Moldova': 4033963,
    'Bosnia and Herzegovina': 3280819, 'Albania': 2877797, 'Lithuania': 2722289,
    'North Macedonia': 2083374, 'Slovenia': 2078938, 'Latvia': 1886198, 'Estonia': 1326535,
    'Montenegro': 628066, 'Luxembourg': 625978, 'Malta': 441543, 'Iceland': 341243,
    'Channel Islands': 173863, 'Isle of Man': 85033, 'Andorra': 77265, 'Faroe Islands': 48863,
    'Monaco': 39242, 'Liechtenstein': 38128, 'San Marino': 33931, 'Gibraltar': 33691,
    'Holy See': 801
}

# Total population of European countries
europe_population = sum(country_populations.values())

# Generate country values with the desired distribution
country_values = []
total_users = len(users_df)

for _ in range(total_users):
    country = np.random.choice(list(country_populations.keys()), p=[pop / europe_population for pop in country_populations.values()])
    country_values.append(country)

# Assign country values to users DataFrame
users_df['country'] = country_values

country_to_region = {
    'Belarus': 'Eastern Europe',
    'Bulgaria': 'Eastern Europe',
    'Czech Republic': 'Eastern Europe',
    'Hungary': 'Eastern Europe',
    'Poland': 'Eastern Europe',
    'Moldova': 'Eastern Europe',
    'Romania': 'Eastern Europe',
    'Russia': 'Eastern Europe',
    'Slovakia': 'Eastern Europe',
    'Ukraine': 'Eastern Europe',
    'Åland Islands': 'Northern Europe',
    'Denmark': 'Northern Europe',
    'Estonia': 'Northern Europe',
    'Faroe Islands': 'Northern Europe',
    'Finland': 'Northern Europe',
    'Iceland': 'Northern Europe',
    'Ireland': 'Northern Europe',
    'Isle of Man': 'Northern Europe',
    'Latvia': 'Northern Europe',
    'Lithuania': 'Northern Europe',
    'Norway': 'Northern Europe',
    'Svalbard and Jan Mayen Islands': 'Northern Europe',
    'Sweden': 'Northern Europe',
    'United Kingdom': 'Northern Europe',
    'Albania': 'Southern Europe',
    'Andorra': 'Southern Europe',
    'Bosnia and Herzegovina': 'Southern Europe',
    'Croatia': 'Southern Europe',
    'Gibraltar': 'Southern Europe',
    'Greece': 'Southern Europe',
    'Holy See': 'Southern Europe',
    'Italy': 'Southern Europe',
    'Malta': 'Southern Europe',
    'Montenegro': 'Southern Europe',
    'North Macedonia': 'Southern Europe',
    'Portugal': 'Southern Europe',
    'San Marino': 'Southern Europe',
    'Serbia': 'Southern Europe',
    'Slovenia': 'Southern Europe',
    'Spain': 'Southern Europe',
    'Austria': 'Western Europe',
    'Belgium': 'Western Europe',
    'France': 'Western Europe',
    'Germany': 'Western Europe',
    'Liechtenstein': 'Western Europe',
    'Luxembourg': 'Western Europe',
    'Monaco': 'Western Europe',
    'Netherlands': 'Western Europe',
    'Switzerland': 'Western Europe',
    'Channel Islands': 'Northern Europe'
}

users_df['region'] = users_df['country'].map(country_to_region)
users_df['userId'] = users_df['userId'].astype('uint32')
users_df['sex'] = users_df['sex'].astype('category')
users_df['age'] = users_df['age'].astype('uint8')
users_df['region'] = users_df['region'].astype('category')
users_df['country'] = users_df['country'].astype('category')

users_df

<ipython-input-8-c6f3cd174f83>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_df['sex'] = sex_values
<ipython-input-8-c6f3cd174f83>:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_df['country'] = country_values
<ipython-input-8-c6f3cd174f83>:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

,userId,genres,age,age_group,sex,country,region
0,1,Action,15,Teenagers (13-19),M,Czech Republic,Eastern Europe
1,7,Action,13,Teenagers (13-19),M,Russia,Eastern Europe
2,11,Action,18,Teenagers (13-19),M,France,Western Europe
3,21,Action,15,Teenagers (13-19),M,Germany,Western Europe
4,24,Action,17,Teenagers (13-19),M,Poland,Eastern Europe
...,...,...,...,...,...,...,...
599,505,Horror,22,Young Adults (20-25),M,Poland,Eastern Europe
600,533,Horror,33,Adults (26-35),F,Hungary,Eastern Europe
601,546,Horror,38,Middle-aged Adults (36-45),M,Germany,Western Europe
602,571,Horror,14,Teenagers (13-19),F,Italy,Southern Europe


In [ ]:
users_df.to_csv('users_df_100k.csv', index=True)

In [ ]:
movies_df.to_csv('movies_df_100k.csv', index=True)

#Kolaborativno filtriranje

In [ ]:
merged_df.drop(["genres"], axis=1, inplace=True)

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

user_movie_matrix = merged_df.pivot(index='userId', columns='movieId', values='rating')
user_movie_matrix.fillna(0, inplace=True)
user_movie_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
user_movie_matrix.to_csv('user_movie_matrix_100k.csv', index=True)

In [ ]:
def subtract_mean(row):
    non_zero_values = row[row != 0]
    mean = non_zero_values.mean()
    row[row != 0] -= mean
    return row

df_updated = user_movie_matrix.apply(lambda row: subtract_mean(row), axis=1)

df_updated

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,-0.366379,0.000000,-0.366379,0.0,0.0,-0.366379,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.363636,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,-1.157399,0.000000,0.000000,0.0,0.0,0.000000,-1.157399,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.213904,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,-0.634176,-1.134176,-1.134176,0.0,0.0,0.000000,0.000000,0.0,0.0,0.865824,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import pandas as pd
from tqdm import tqdm

def calculate_pearson_correlation(df):

    def helper_correlation(series_a, series_b):
        return series_a.corr(series_b)

    # Initialize an empty DataFrame to store correlations
    pearson_correlations = pd.DataFrame(index=df.index, columns=df.index)

    for other in tqdm(df.index, desc="Calculating Pearson Correlation"):
        # Get correlations for a single row and store it in the DataFrame
        correlations = df.apply(lambda x: helper_correlation(x, df.loc[other]), axis=1)
        pearson_correlations.loc[other] = correlations

    return pearson_correlations

pearson_correlations = calculate_pearson_correlation(df_updated)

pearson_correlations

Calculating Pearson Correlation: 100%|██████████| 610/610 [02:24<00:00,  4.23it/s]


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.001265,0.000553,0.048419,0.021847,-0.045497,-0.0062,0.047013,0.01951,-0.008754,...,0.018127,-0.017172,-0.015221,-0.037059,-0.029121,0.012016,0.055261,0.075224,-0.025713,0.010932
2,0.001265,1.0,-0.0,-0.017164,0.021796,-0.021051,-0.011114,-0.048085,-0.0,0.003012,...,-0.050551,-0.031581,-0.001688,0.0,0.0,0.006226,-0.020504,-0.006001,-0.060091,0.024999
3,0.000553,-0.0,1.0,-0.01126,-0.031539,0.0048,0.0,-0.032471,-0.0,-0.0,...,-0.004904,-0.016117,0.017749,0.0,-0.001431,-0.037289,-0.007789,-0.013001,0.0,0.01955
4,0.048419,-0.017164,-0.01126,1.0,-0.02962,0.013956,0.058091,0.002065,-0.005874,0.05159,...,-0.037687,0.063122,0.02764,-0.013782,0.040037,0.02059,0.014628,-0.037569,-0.017884,-0.000995
5,0.021847,0.021796,-0.031539,-0.02962,1.0,0.009111,0.010117,-0.012284,0.0,-0.033165,...,0.015964,0.012427,0.027076,0.012461,-0.036272,0.026319,0.031896,-0.001751,0.093829,-0.000278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.012016,0.006226,-0.037289,0.02059,0.026319,-0.009137,0.028326,0.022277,0.031633,-0.039946,...,0.053683,0.016384,0.098011,0.061078,0.019678,1.0,0.017927,0.056676,0.038422,0.075464
607,0.055261,-0.020504,-0.007789,0.014628,0.031896,0.045501,0.030981,0.048822,-0.012161,-0.017656,...,0.049059,0.038197,0.049317,0.002355,-0.029381,0.017927,1.0,0.044514,0.019049,0.02186
608,0.075224,-0.006001,-0.013001,-0.037569,-0.001751,0.021727,0.028414,0.071759,0.032783,-0.052,...,0.069198,0.051388,0.012801,0.006319,-0.007978,0.056676,0.044514,1.0,0.050714,0.054454


In [ ]:
pearson_correlations

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.001265,0.000553,0.048419,0.021847,-0.045497,-0.0062,0.047013,0.01951,-0.008754,...,0.018127,-0.017172,-0.015221,-0.037059,-0.029121,0.012016,0.055261,0.075224,-0.025713,0.010932
2,0.001265,1.0,-0.0,-0.017164,0.021796,-0.021051,-0.011114,-0.048085,-0.0,0.003012,...,-0.050551,-0.031581,-0.001688,0.0,0.0,0.006226,-0.020504,-0.006001,-0.060091,0.024999
3,0.000553,-0.0,1.0,-0.01126,-0.031539,0.0048,0.0,-0.032471,-0.0,-0.0,...,-0.004904,-0.016117,0.017749,0.0,-0.001431,-0.037289,-0.007789,-0.013001,0.0,0.01955
4,0.048419,-0.017164,-0.01126,1.0,-0.02962,0.013956,0.058091,0.002065,-0.005874,0.05159,...,-0.037687,0.063122,0.02764,-0.013782,0.040037,0.02059,0.014628,-0.037569,-0.017884,-0.000995
5,0.021847,0.021796,-0.031539,-0.02962,1.0,0.009111,0.010117,-0.012284,0.0,-0.033165,...,0.015964,0.012427,0.027076,0.012461,-0.036272,0.026319,0.031896,-0.001751,0.093829,-0.000278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.012016,0.006226,-0.037289,0.02059,0.026319,-0.009137,0.028326,0.022277,0.031633,-0.039946,...,0.053683,0.016384,0.098011,0.061078,0.019678,1.0,0.017927,0.056676,0.038422,0.075464
607,0.055261,-0.020504,-0.007789,0.014628,0.031896,0.045501,0.030981,0.048822,-0.012161,-0.017656,...,0.049059,0.038197,0.049317,0.002355,-0.029381,0.017927,1.0,0.044514,0.019049,0.02186
608,0.075224,-0.006001,-0.013001,-0.037569,-0.001751,0.021727,0.028414,0.071759,0.032783,-0.052,...,0.069198,0.051388,0.012801,0.006319,-0.007978,0.056676,0.044514,1.0,0.050714,0.054454


In [ ]:
pearson_correlations.to_csv('correlation_matrix_100k.csv', index=True)

In [ ]:
pearson_correlations.head()
correlation_df=pearson_correlations

In [ ]:
movies_df.loc[movies_df['movieId']==4993]

,movieId,title,genres
3638,4993,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy


In [ ]:
def get_most_similar_users(user_id, n, correlation_df):
    user_correlations = correlation_df[user_id].drop(user_id)
    sorted_users = user_correlations.sort_values(ascending=False)
    top_n_similar_users = sorted_users.head(n).index.tolist()
    return top_n_similar_users

def get_movie_recommendations(user_id, m, similar_users, user_movie_matrix, min_correlation=0.05):
    user_index = user_movie_matrix.index.get_loc(user_id)

    # Sum the correlations (weights) and weighted scores of similar users
    weights_sum = 0
    weighted_scores_sum = np.zeros_like(user_movie_matrix.loc[user_id])

    for similar_user in similar_users:
        similar_user_index = user_movie_matrix.index.get_loc(similar_user)
        correlation = correlation_df.at[user_id, similar_user]

        if np.isnan(correlation) or correlation < min_correlation:
            continue

        weights_sum += correlation
        weighted_scores_sum += user_movie_matrix.loc[similar_user] * correlation

    # Normalize the weighted scores by dividing by the sum of correlations
    if weights_sum != 0:
        normalized_scores = weighted_scores_sum / weights_sum
    else:
        normalized_scores = np.zeros_like(user_movie_matrix.loc[user_id])

    # Remove the movies that the user has already rated
    user_rated_movies = user_movie_matrix.loc[user_id]
    normalized_scores[user_rated_movies != 0] = 0

    # Get the top m movie recommendations
    # Get a pandas Series with the movie indices as the index
    normalized_scores_series = pd.Series(normalized_scores, index=user_movie_matrix.columns)
    # Select the top m movie recommendations by using the nlargest function on the pandas Series
    recommended_movie_ids = normalized_scores_series.nlargest(m).index
    return recommended_movie_ids

user_id = 2  # Input your user_id
n = 5  # Input the number of similar users
m = 10  # Input the number of movie recommendations

# Get the top n most similar users
similar_users = get_most_similar_users(user_id, n, correlation_df)
print(f"Most similar users to user {user_id}: {similar_users}")

# Get the top m movie recommendations for the user_id using similar_users
recommended_movie_ids = get_movie_recommendations(user_id, m, similar_users, user_movie_matrix, min_correlation=0.05)
print(f"Top {m} movie recommendations for user {user_id}: {recommended_movie_ids}")

# Get movie titles from movie ids
recommended_movie_titles = movies_df[movies_df['movieId'].isin(recommended_movie_ids)]['title']
print("Recommended movie titles:")
print(recommended_movie_titles)

Most similar users to user 2: [189, 246, 378, 209, 227]
Top 10 movie recommendations for user 2: UInt64Index([4993, 5952, 2571, 7147, 60684, 7153, 68954, 6377, 48780, 70286], dtype='uint64', name='movieId')
Recommended movie titles:
1939                                   Matrix, The (1999)
3638    Lord of the Rings: The Fellowship of the Ring,...
4137        Lord of the Rings: The Two Towers, The (2002)
4360                                  Finding Nemo (2003)
4796                                      Big Fish (2003)
4800    Lord of the Rings: The Return of the King, The...
6331                                 Prestige, The (2006)
6797                                      Watchmen (2009)
7039                                            Up (2009)
7090                                    District 9 (2009)
Name: title, dtype: object


In [ ]:
target_users = [189, 246, 378, 209, 227]

for user_id in target_users:
    # Get the top n most similar users
    similar_users = get_most_similar_users(user_id, n, correlation_df)

    # Get the top m movie recommendations for the user_id using similar_users
    recommended_movie_ids = get_movie_recommendations(user_id, m, similar_users, user_movie_matrix, min_correlation=0.05)

    # Get movie titles from movie ids
    recommended_movie_titles = movies_df[movies_df['movieId'].isin(recommended_movie_ids)]['title']

    print(f"User {user_id}:")
    print(f"Most similar users to user {user_id}: {similar_users}")
    print(f"Top {m} movie recommendations for user {user_id}: {recommended_movie_ids}")
    print("Recommended movie titles:")
    print(recommended_movie_titles)
    print("--------------------------------------------------")

User 189:
Most similar users to user 189: [60, 319, 2, 65, 196]
Top 10 movie recommendations for user 189: UInt64Index([858, 364, 60756, 80906, 106782, 122882, 131724, 89774, 150, 733], dtype='uint64', name='movieId')
Recommended movie titles:
123                                      Apollo 13 (1995)
322                                 Lion King, The (1994)
592                                      Rock, The (1996)
659                                 Godfather, The (1972)
6801                                 Step Brothers (2008)
7436                                    Inside Job (2010)
7697                                       Warrior (2011)
8305                      Wolf of Wall Street, The (2013)
8681                            Mad Max: Fury Road (2015)
8828    The Jinx: The Life and Deaths of Robert Durst ...
Name: title, dtype: object
--------------------------------------------------
User 246:
Most similar users to user 246: [332, 2, 72, 120, 601]
Top 10 movie recommendations for 

In [ ]:
import random

top_n_rated_movies = 15
top_m_recommendations = 8

# Function to get the top N rated movies for a user
def get_top_rated_movies(user_id, n, user_movie_matrix):
    rated_movies = user_movie_matrix.loc[user_id].sort_values(ascending=False)
    top_rated_movies = rated_movies.head(n).index
    return top_rated_movies

random_users = np.random.choice(user_movie_matrix.index, size=10, replace=False)

for user_id in random_users:
    # Get the top N rated movies for the user
    top_rated_movie_ids = get_top_rated_movies(user_id, top_n_rated_movies, user_movie_matrix)
    top_rated_movie_titles = movies_df[movies_df['movieId'].isin(top_rated_movie_ids)]['title']

    # Get the top M recommendations for the user
    similar_users = get_most_similar_users(user_id, n, correlation_df)
    recommended_movie_ids = get_movie_recommendations(user_id, top_m_recommendations, similar_users, user_movie_matrix, min_correlation=0.05)
    recommended_movie_titles = movies_df[movies_df['movieId'].isin(recommended_movie_ids)]['title']

    print(f"User {user_id}:")
    print(f"Top {top_n_rated_movies} rated movies:")
    print(top_rated_movie_titles)
    print("--------------------------------------------------")
    print(f"Top {top_m_recommendations} movie recommendations:")
    print(recommended_movie_titles)
    print("==================================================")

User 409:
Top 15 rated movies:
257                            Pulp Fiction (1994)
692                        Some Like It Hot (1959)
701                       Wizard of Oz, The (1939)
914                              Goodfellas (1990)
922                 Godfather: Part II, The (1974)
975                          Cool Hand Luke (1967)
1529                             Roger & Me (1989)
1960                               Election (1999)
2101            National Lampoon's Vacation (1983)
2145                        American Beauty (1999)
2515                      American Graffiti (1973)
2524    Taking of Pelham One Two Three, The (1974)
2555                           Animal House (1978)
2733                        Blazing Saddles (1974)
2852                         Love and Death (1975)
Name: title, dtype: object
--------------------------------------------------
Top 8 movie recommendations:
461                               Schindler's List (1993)
602     Dr. Strangelove or: How I Learn

#Cold start

In [ ]:
def get_new_user(new_index):
    age_distribution = users_df['age'].value_counts(normalize=True)
    sex_distribution = users_df['sex'].value_counts(normalize=True)
    region_distribution = users_df['region'].value_counts(normalize=True)
    country_distribution = users_df['country'].value_counts(normalize=True)

    new_age = np.random.choice(age_distribution.index, p=age_distribution.values)
    new_sex = np.random.choice(sex_distribution.index, p=sex_distribution.values)
    new_country = np.random.choice(country_distribution.index, p=country_distribution.values)

    new_region = country_to_region[new_country]
    new_age_group = pd.cut([new_age], bins=[12, 19, 25, 35, 45, 65], labels=list(age_ranges))


    new_user = pd.DataFrame({
        'userId': [new_index],
        'sex': [new_sex],
        'age': [new_age],
        'country': [new_country],
        'region': [new_region]
    })

    new_user['age_group_pom'] = pd.cut(new_user['age'], bins=[13, 19, 25, 35, 45, 65], labels=list(age_ranges))
    new_user.insert(new_user.columns.get_loc('age') + 1, 'age_group', new_user['age_group_pom'])

    return new_user.drop('age_group_pom', axis=1)

In [ ]:
get_new_user(170000)

,userId,sex,age,age_group,country,region
0,170000,F,34,Adults (26-35),Poland,Eastern Europe


In [ ]:
from sklearn.metrics.pairwise import cosine_distances
from sklearn.preprocessing import LabelEncoder
from collections import OrderedDict


min_age = 13
max_age = 65

def normalized_difference(a, b, min_age, max_age):
    return np.abs((a - min_age) - (b - min_age)) / (max_age - min_age)

def find_similar_users(users_df, new_user, n):
    # Get new user details
    new_sex = new_user['sex'].item()
    new_age_group = new_user['age_group'].item()
    new_age = new_user['age'].item()
    new_country = new_user['country'].item()
    new_region = new_user['region'].item()

    # Compute age distance range
    min_age = users_df['age'].min()
    max_age = users_df['age'].max()

    # Calculate age group similarity
    age_distance = normalized_difference(users_df['age'], new_age, min_age, max_age)
    age_similarity = 0.45 * (1 - age_distance)

    # Calculate overall similarity
    similarity_scores = (
        (users_df['sex'] == new_sex).astype(float) * 0.35 +
        (users_df['age_group'] == new_age_group).astype(float) * age_similarity +
        (users_df['country'] == new_country).astype(float) * 0.15 +
        (users_df['region'] == new_region).astype(float) * 0.05
    )

    # Sort users by similarity scores and get top n
    top_indices = np.argsort(similarity_scores)[::-1][:n]
    top_similarities = similarity_scores[top_indices]

    # Get top n similar users
    similar_users = users_df.loc[top_indices]

    return similar_users, top_similarities

In [ ]:
import time

# Function 1
new_user=get_new_user(170000)
start_time = time.time()
similar_users, top_similarities=find_similar_users(users_df, new_user, 10)
elapsed_time_1 = time.time() - start_time
new_user
similar_users
# Call the first implementation here

print(elapsed_time_1)
print(similar_users)
print(new_user)

0.004522562026977539
     userId  genres  age                   age_group sex      country  \
462     273  Comedy   44  Middle-aged Adults (36-45)   M      Germany   
354     502   Drama   44  Middle-aged Adults (36-45)   M      Germany   
172     146   Drama   45  Middle-aged Adults (36-45)   M  Netherlands   
312     423   Drama   45  Middle-aged Adults (36-45)   M      Germany   
320     435   Drama   44  Middle-aged Adults (36-45)   M      Hungary   
299     405   Drama   44  Middle-aged Adults (36-45)   M      Ukraine   
244     300   Drama   44  Middle-aged Adults (36-45)   M       Sweden   
365     521   Drama   44  Middle-aged Adults (36-45)   M        Spain   
401     599   Drama   44  Middle-aged Adults (36-45)   M      Ukraine   
520     538  Comedy   45  Middle-aged Adults (36-45)   M       Russia   

              region  
462   Western Europe  
354   Western Europe  
172   Western Europe  
312   Western Europe  
320   Eastern Europe  
299   Eastern Europe  
244  Northern 

In [ ]:
def get_movie_recommendations(m, similar_users, similarities, user_movie_matrix, min_correlation=0.35):

    weights_sum = 0
    weighted_scores_sum = np.zeros_like(user_movie_matrix.loc[similar_users[0]])

    for similar_user, similarity in zip(similar_users, similarities):
        similar_user_index = user_movie_matrix.index.get_loc(similar_user)

        if np.isnan(similarity) or similarity < min_correlation:
            continue

        weights_sum += similarity
        weighted_scores_sum += user_movie_matrix.loc[similar_user] * similarity

    # Normalize the weighted scores by dividing by the sum of correlations
    if weights_sum != 0:
        normalized_scores = weighted_scores_sum / weights_sum
    else:
        normalized_scores = np.zeros_like(user_movie_matrix.loc[similar_user[0]])


    # Get the top m movie recommendations
    # Get a pandas Series with the movie indices as the index
    normalized_scores_series = pd.Series(normalized_scores, index=user_movie_matrix.columns)
    # Select the top m movie recommendations by using the nlargest function on the pandas Series
    recommended_movie_ids = normalized_scores_series.nlargest(m).index
    return recommended_movie_ids

n = 5  # Input the number of similar users
m = 10  # Input the number of movie recommendations
new_index=1000

new_user=get_new_user(new_index)
print(new_user, "\n")
similar_users, similarities = find_similar_users(users_df, new_user, n)
print(f"Most similar users to user new user: {similar_users}")
print()
# Get the top m movie recommendations for the user_id using similar_users
recommended_movie_ids = get_movie_recommendations( m, similar_users['userId'].tolist(), similarities, user_movie_matrix, min_correlation=0.05)
print(f"Top {m} movie recommendations for new user: {recommended_movie_ids}")
print()
# Get movie titles from movie ids
recommended_movie_titles = movies_df[movies_df['movieId'].isin(recommended_movie_ids)]['title']
print("Recommended movie titles:")
print(recommended_movie_titles)

   userId sex  age          age_group  country          region
0    1000   M   15  Teenagers (13-19)  Germany  Western Europe 

Most similar users to user new user:      userId  genres  age          age_group sex  country          region
23      112  Action   15  Teenagers (13-19)   M  Germany  Western Europe
512     508  Comedy   15  Teenagers (13-19)   M  Germany  Western Europe
3        21  Action   15  Teenagers (13-19)   M  Germany  Western Europe
70      407  Action   15  Teenagers (13-19)   M  Germany  Western Europe
333     460   Drama   16  Teenagers (13-19)   M  Germany  Western Europe

Top 10 movie recommendations for new user: UInt64Index([1270, 2011, 2012, 2571, 1210, 260, 1196, 58559, 1222, 480], dtype='uint64', name='movieId')

Recommended movie titles:
224             Star Wars: Episode IV - A New Hope (1977)
418                                  Jurassic Park (1993)
898     Star Wars: Episode V - The Empire Strikes Back...
911     Star Wars: Episode VI - Return of the J